In [1]:
#import libraries
import functions as func
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [13]:
#read data
df = pd.read_csv("houseprices.csv")

df = func.tidyData(df)

In [17]:
y = df["SalePrice"].tolist()
X = df.copy(deep=True)
X.drop(["SalePrice"], axis=1, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
clf = LogisticRegression(random_state=3)
clf.fit(X_train, y_train)
y_pred = clf.predict_proba(X_test)
y_pred

ValueError: could not convert string to float: 'Pave'